In [1]:
import pandas as pd
import os
import tensorflow as tf

import scipy
from scipy.io import loadmat
import re

import string
import imageio
import numpy as np
import matplotlib.pyplot as plt
import logging, scipy, multiprocessing
import matplotlib.image as img
import random
import time
import nltk
import cv2
import warnings
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import *
warnings.filterwarnings('ignore')

# Load Data

In [2]:
'''
train_hr_img_list = sorted(tl.files.load_file_list(path=".\image_2", regx='.*.png', printable=False))
train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=".\image_2", n_threads=32)
'''

'\ntrain_hr_img_list = sorted(tl.files.load_file_list(path=".\\image_2", regx=\'.*.png\', printable=False))\ntrain_hr_imgs = tl.vis.read_images(train_hr_img_list, path=".\\image_2", n_threads=32)\n'

In [3]:
def generator_train():
        for img in train_hr_imgs:
            yield img
def _map_fn_train(img):
        hr_patch = tf.image.random_crop(img, [256, 256, 3])
        hr_patch = hr_patch / (255. / 2.)
        hr_patch = hr_patch - 1.
        hr_patch = tf.image.random_flip_left_right(hr_patch)
        lr_patch = tf.image.resize_images(hr_patch, (64, 64), method=2)
        #lr_patch = tf.image.resize(hr_patch, size=[64, 64])
        return lr_patch, hr_patch
def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image
    return img

In [4]:
def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def save_images(images, size, image_path):
    return imsave(images, size, image_path)

In [5]:

train_ds = tf.data.Dataset.from_generator(generator_train, output_types=(tf.float32))
train_ds = train_ds.map(_map_fn_train, num_parallel_calls=multiprocessing.cpu_count())
train_ds = train_ds.repeat()
train_ds = train_ds.shuffle(3*9)
train_ds = train_ds.batch(9)


# Model

In [6]:
def deconv2d(input_, output_shape,
             k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
             name="deconv2d"):
    with tf.variable_scope(name):
        
        w = tf.get_variable('w', [k_h, k_h, output_shape[-1], input_.get_shape()[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))


        deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,strides=[1, d_h, d_w, 1])



        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())
        
        return deconv

In [7]:
class Generator:
    def __init__(self, noise_z, training_phase, hparas, reuse):
        self.z = noise_z
        self.train = training_phase
        self.hparas = hparas
        self.reuse = reuse
        self._build_model()
    def _build_model(self):
        """ Generator in Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network
        feature maps (n) and stride (s) feature maps (n) and stride (s)
        """
        relu = tf.nn.relu
        tanh = tf.nn.tanh
        conv2d = tf.layers.conv2d
        bn = tf.layers.batch_normalization
        linear = tf.layers.dense
        with tf.variable_scope("generator", reuse=self.reuse) as vs:
            n = relu(conv2d(self.z, 64, kernel_size=3, strides=1, padding='same'))
            temp = n
            # B residual blocks
            for i in range(16):
                nn = conv2d(n, 64, kernel_size=3, strides=1, padding='same')
                nn = relu(bn(nn,training = self.train ))
                nn = conv2d(nn, 64, kernel_size=3, strides=1, padding='same')
                nn = bn(nn,training = self.train )
                nn = tf.add(n,nn)
            n = conv2d(n, 64, kernel_size=3, strides=1, padding='same')
            n = bn(nn,training = self.train )
            n = tf.add(n,temp)
            # B residual blacks end
            n = conv2d(n, 256, kernel_size=3, strides=1, padding='same')
            n = relu(deconv2d(n, [9, 128, 128 , 256], name='g_h1'))
            n = conv2d(n, 256, kernel_size=3, strides=1, padding='same')
            n = relu(deconv2d(n, [9, 256, 256 , 256], name='g_h2'))
            n = tanh(conv2d(n, 3, kernel_size=3, strides=1, padding='same'))
            self.outputs = n

In [8]:
# resnet structure
class Discriminator:
    def __init__(self, image, training_phase, hparas, reuse):
        self.image = image
        self.train = training_phase
        self.hparas = hparas
        self.reuse = reuse
        self._build_model()
    
    def _build_model(self): 
        lrelu = tf.nn.leaky_relu
        conv2d = tf.layers.conv2d
        bn = tf.layers.batch_normalization
        dense = tf.layers.dense
        df_dim = 64
        with tf.variable_scope('discriminator', reuse=self.reuse):
            n = lrelu(conv2d(self.image, df_dim, kernel_size=4, strides=2, padding='same'))
            n = lrelu(bn(conv2d(n, df_dim*2 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*4 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*8 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*16 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*32 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*16 , kernel_size=1, strides=1, padding='same'),training = self.train))
            nn = bn(conv2d(n, df_dim*8 , kernel_size=1, strides=1, padding='same'),training = self.train)
            n = lrelu(bn(conv2d(nn, df_dim*2 , kernel_size=1, strides=1, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*2 , kernel_size=3, strides=1, padding='same'),training = self.train))
            n = bn(conv2d(n, df_dim*8 , kernel_size=3, strides=1, padding='same'),training = self.train)
            n = tf.add(n,nn)
            n = tf.layers.Flatten()(n)
            d_net = dense(n,1)
            self.logits = d_net
            #self.discriminator_net = d_net
            self.outputs =  d_net

In [9]:
def Vgg19_simple_api(rgb, reuse):
    """
    Build the VGG 19 Model
    Parameters
    -----------
    rgb : rgb image placeholder [batch, height, width, 3] values scaled [0, 1]
    """
    VGG_MEAN = [103.939, 116.779, 123.68]
    with tf.variable_scope("VGG19", reuse=reuse) as vs:
        start_time = time.time()
        print("build model started")
        rgb_scaled = rgb * 255.0
        # Convert RGB to BGR
        if tf.__version__ <= '0.11':
            red, green, blue = tf.split(3, 3, rgb_scaled)
        else: # TF 1.0
            # print(rgb_scaled)
            red, green, blue = tf.split(rgb_scaled, 3, 3)
        assert red.get_shape().as_list()[1:] == [224, 224, 1]
        assert green.get_shape().as_list()[1:] == [224, 224, 1]
        assert blue.get_shape().as_list()[1:] == [224, 224, 1]
        if tf.__version__ <= '0.11':
            bgr = tf.concat(3, [
                blue - VGG_MEAN[0],
                green - VGG_MEAN[1],
                red - VGG_MEAN[2],
            ])
        else:
            bgr = tf.concat([
                blue - VGG_MEAN[0],
                green - VGG_MEAN[1],
                red - VGG_MEAN[2],
            ], axis=3)
        assert bgr.get_shape().as_list()[1:] == [224, 224, 3]

        """ input layer """
        net_in = InputLayer(bgr, name='input')
        """ conv1 """
        network = Conv2d(net_in, n_filter=64, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv1_1')
        network = Conv2d(network, n_filter=64, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv1_2')
        network = MaxPool2d(network, filter_size=(2, 2), strides=(2, 2),
                    padding='SAME', name='pool1')
        """ conv2 """
        network = Conv2d(network, n_filter=128, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv2_1')
        network = Conv2d(network, n_filter=128, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv2_2')
        network = MaxPool2d(network, filter_size=(2, 2), strides=(2, 2),
                    padding='SAME', name='pool2')
        """ conv3 """
        network = Conv2d(network, n_filter=256, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv3_1')
        network = Conv2d(network, n_filter=256, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv3_2')
        network = Conv2d(network, n_filter=256, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv3_3')
        network = Conv2d(network, n_filter=256, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv3_4')
        network = MaxPool2d(network, filter_size=(2, 2), strides=(2, 2),
                    padding='SAME', name='pool3')
        """ conv4 """
        network = Conv2d(network, n_filter=512, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv4_1')
        network = Conv2d(network, n_filter=512, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv4_2')
        network = Conv2d(network, n_filter=512, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv4_3')
        network = Conv2d(network, n_filter=512, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv4_4')
        network = MaxPool2d(network, filter_size=(2, 2), strides=(2, 2),
                    padding='SAME', name='pool4')                               # (batch_size, 14, 14, 512)
        conv = network
        """ conv5 """
        network = Conv2d(network, n_filter=512, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv5_1')
        network = Conv2d(network, n_filter=512, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv5_2')
        network = Conv2d(network, n_filter=512, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv5_3')
        network = Conv2d(network, n_filter=512, filter_size=(3, 3),
                    strides=(1, 1), act=tf.nn.relu,padding='SAME', name='conv5_4')
        network = MaxPool2d(network, filter_size=(2, 2), strides=(2, 2),
                    padding='SAME', name='pool5')                               # (batch_size, 7, 7, 512)
        """ fc 6~8 """
        network = FlattenLayer(network, name='flatten')
        network = DenseLayer(network, n_units=4096, act=tf.nn.relu, name='fc6')
        network = DenseLayer(network, n_units=4096, act=tf.nn.relu, name='fc7')
        network = DenseLayer(network, n_units=1000, act=tf.identity, name='fc8')
        print("build model finished: %fs" % (time.time() - start_time))
        return network, conv

In [10]:
def get_hparas():
    hparas = {
        'BATCH_SIZE' : 9,
        'LR' : 1e-4,
        'BETA' : 0.9, # AdamOptimizer parameter
        'N_EPOCH' : 100,
        'N_SAMPLE' : 7480
    }
    return hparas

In [11]:
class GAN:
    def __init__(self, hparas, training_phase, ckpt_path, inference_path, test_img = None,recover=None,savename = None):
        self.hparas = hparas
        self.train = training_phase
        self.ckpt_path = ckpt_path
        self.sample_path = './samples'
        self.inference_path = './inference'
        
        self._get_session() # get session
        self._get_train_data_iter() # initialize and get data iterator
        self._input_layer() # define input placeholder
        self._get_inference() # build generator and discriminator
        self._get_loss() # define gan loss
        self._get_var_with_name() # get variables for each part of model
        self._optimize() # define optimizer
        self._init_vars()
        self._get_saver()
        self.test_LRbatch = test_img
        self.savename = savename
        if recover is not None:
            self._load_checkpoint(recover)
            #self.test_LRbatch = test_img
            
            
        #print (self.generator  )  
        
    def _get_train_data_iter(self):
        if self.train: # training data iteratot
            iterator_oneshot = train_ds.make_one_shot_iterator()
            self.next_element_oneshot = iterator_oneshot.get_next()
           
            
    def _input_layer(self):
        if self.train:
            self.real_image = tf.placeholder(tf.float32,[None, 256,256, 3],name='real_image')
            self.z_noise = tf.placeholder(tf.float32, [None, 64,64,3], name='z_noise')
        else:
            
            self.z_noise = tf.placeholder(tf.float32, [None, 64,64,3], name='z_noise')
    
    def _get_inference(self):
        if self.train:
            
            # generating image
            generator = Generator(self.z_noise, training_phase=True,hparas=self.hparas, reuse=False)
            self.generator = generator
            # discriminize
            # fake image
            fake_discriminator = Discriminator(generator.outputs,training_phase=True, hparas=self.hparas, reuse=False)
            self.fake_discriminator = fake_discriminator
            # real image
            real_discriminator = Discriminator(self.real_image, training_phase=True,hparas=self.hparas, reuse=True)
            self.real_discriminator = real_discriminator
            '''VGG'''
            t_target_image_224 = tf.image.resize_images(self.real_image, size=[224, 224], method=0, align_corners=False)
            t_predict_image_224 = tf.image.resize_images(generator.outputs, size=[224, 224], method=0, align_corners=False)
            self.net_vgg, self.vgg_target_emb = Vgg19_simple_api((t_target_image_224+1)/2, reuse=False)
            _, self.vgg_predict_emb = Vgg19_simple_api((t_predict_image_224+1)/2, reuse=True)

            
        else: # inference mode
            self.generate_image_net = Generator(self.z_noise, training_phase=True,hparas=self.hparas, reuse=False)
            
    def _get_loss(self):
        if self.train:
            d_loss1 = tl.cost.sigmoid_cross_entropy(self.real_discriminator.logits, tf.ones_like(self.real_discriminator.logits), name='d1')
            d_loss2 = tl.cost.sigmoid_cross_entropy(self.fake_discriminator.logits, tf.zeros_like(self.fake_discriminator.logits), name='d2')
            self.d_loss = d_loss1 + d_loss2
            mse_loss = tl.cost.mean_squared_error(self.generator.outputs, self.real_image, is_mean=True)
            self.initmse_loss = mse_loss
            g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(self.fake_discriminator.logits, tf.ones_like(self.fake_discriminator.logits), name='g')
            vgg_loss = 2e-6 * tl.cost.mean_squared_error(self.vgg_predict_emb.outputs, self.vgg_target_emb.outputs, is_mean=True)
            self.g_loss = mse_loss + g_gan_loss + vgg_loss
    
    def _optimize(self):
        if self.train:
            with tf.variable_scope('learning_rate'):
                self.lr_var = tf.Variable(self.hparas['LR'], trainable=False)

            discriminator_optimizer = tf.train.AdamOptimizer(self.lr_var, beta1=self.hparas['BETA'])
            generator_optimizer = tf.train.AdamOptimizer(self.lr_var, beta1=self.hparas['BETA'])
            g_optim_init_optimizer = tf.train.AdamOptimizer(self.lr_var, beta1=self.hparas['BETA'])
            
            self.d_optim = discriminator_optimizer.minimize(self.d_loss, var_list=self.discrim_vars)
            self.g_optim = generator_optimizer.minimize(self.g_loss, var_list=self.generator_vars)
            self.g_optim_init = g_optim_init_optimizer.minimize(self.initmse_loss, var_list=self.generator_vars)
        
    def training(self):
        '''VGG'''
        vgg19_npy_path = "./vgg19.npy"
        if not os.path.isfile(vgg19_npy_path):
            print("Please download vgg19.npz from : https://github.com/machrisaa/tensorflow-vgg")
            exit()
        npz = np.load(vgg19_npy_path, encoding='latin1').item()

        params = []
        for val in sorted( npz.items() ):
            W = np.asarray(val[1][0])
            b = np.asarray(val[1][1])
            print("  Loading %s: %s, %s" % (val[0], W.shape, b.shape))
            params.extend([W, b])
        tl.files.assign_params(self.sess, params, self.net_vgg)
        # net_vgg.print_params(False)
        # net_vgg.print_layers()

        for _epoch in range(self.hparas['N_EPOCH']):
            start_time = time.time()
            n_batch_epoch = int(self.hparas['N_SAMPLE']/self.hparas['BATCH_SIZE'])
            if _epoch < 5:
                for _step in range(n_batch_epoch):
                    step_time = time.time()
                    lr_batch, hr_batch = self.sess.run(self.next_element_oneshot)
                    self.lr_batch = lr_batch
                    self.hr_batch = hr_batch
                    self.generator_error, _ = self.sess.run([self.initmse_loss, self.g_optim_init],
                                                           feed_dict={self.z_noise:self.lr_batch ,
                                                                      self.real_image:self.hr_batch})
                    if _step%50==0:
                        print("Epoch: [%2d/%2d] [%4d/%4d] time: %4.4fs, g_loss: %.3f" \
                                % (_epoch, self.hparas['N_EPOCH'], _step, n_batch_epoch, time.time() - step_time,
                                    self.generator_error))
            else:
                for _step in range(n_batch_epoch):
                    step_time = time.time()
                    lr_batch, hr_batch = self.sess.run(self.next_element_oneshot)
                    self.lr_batch = lr_batch
                    self.hr_batch = hr_batch
                    # update discriminator
                    self.discriminator_error, _ = self.sess.run([self.d_loss, self.d_optim],
                                                                   feed_dict={
                                                                        self.real_image:self.hr_batch,
                                                                        self.z_noise:self.lr_batch})

                    # update generate
                    self.generator_error, _ = self.sess.run([self.g_loss, self.g_optim],
                                                           feed_dict={self.z_noise:self.lr_batch ,
                                                                      self.real_image:self.hr_batch})


                    if _step%50==0:
                        print("Epoch: [%2d/%2d] [%4d/%4d] time: %4.4fs, d_loss: %.3f, g_loss: %.3f" \
                                % (_epoch, self.hparas['N_EPOCH'], _step, n_batch_epoch, time.time() - step_time,
                                   self.discriminator_error, self.generator_error))
                    
                self._save_checkpoint(_epoch)
            self._sample_visiualize(_epoch)
            #self._sample_visiualize(_epoch)
            
    def inference(self):
        ni = int(np.ceil(np.sqrt(self.hparas['BATCH_SIZE'])))
        img_gen = self.sess.run(self.generate_image_net.outputs, feed_dict={self.z_noise : self.test_LRbatch})
        save_images(img_gen, [ni, ni], './test_img/SR'+self.savename+'.png')
        print('-----success save SRcrop--------')
        
    def _init_vars(self):
        self.sess.run(tf.global_variables_initializer())
    
    def _get_session(self):
        self.sess = tf.Session()
    
    def _get_saver(self):
        if self.train:
            self.g_saver = tf.train.Saver(var_list=self.generator_vars)
            self.d_saver = tf.train.Saver(var_list=self.discrim_vars)
        else:
            self.g_saver = tf.train.Saver(var_list=self.generator_vars)
            
    def _sample_visiualize(self, epoch):
        ni = int(np.ceil(np.sqrt(self.hparas['BATCH_SIZE'])))
        img_gen = self.sess.run(self.generator.outputs,feed_dict={self.z_noise : self.test_LRbatch})
        save_images(img_gen, [ni, ni], './SR_training/Gentrain_{:02d}.png'.format(epoch))
        
    def _get_var_with_name(self):
        t_vars = tf.trainable_variables()
        self.generator_vars = [var for var in t_vars if 'generator' in var.name]
        self.discrim_vars = [var for var in t_vars if 'discriminator' in var.name]
    
    def _load_checkpoint(self, recover):
        if self.train:
            self.g_saver.restore(self.sess, self.ckpt_path+'g_model_'+str(recover)+'.ckpt')
            self.d_saver.restore(self.sess, self.ckpt_path+'d_model_'+str(recover)+'.ckpt')
        else:
            self.g_saver.restore(self.sess, self.ckpt_path+'g_model_'+str(recover)+'.ckpt')
        print('-----success restored checkpoint--------')
    
    def _save_checkpoint(self, epoch):
        self.g_saver.save(self.sess, self.ckpt_path+'g_model_'+str(epoch)+'.ckpt')
        self.d_saver.save(self.sess, self.ckpt_path+'d_model_'+str(epoch)+'.ckpt')
        print('-----success saved checkpoint--------')

# Training

In [12]:

from PIL import Image
crop_img = Image.open('./000057_3.png')
#plt.imshow(crop_img)
testLR_img = []
width, height = crop_img.size
start_pos = start_x, start_y = (0, 0)
cropped_image_size = w, h = (64, 64)

for row_i in range(0, width, w):
    for col_i in range(0, height, h):
        lr = crop_img.crop((col_i, row_i, col_i + w, row_i + h))
        pic = np.array(lr)
        testLR_img.append(pic)
LRimgs = np.array(testLR_img)
print(LRimgs.shape)
LRimgs = LRimgs / (255. / 2.)
LRimgs = LRimgs - 1.
#print(LRimgs)


(9, 64, 64, 3)


In [ ]:

tf.reset_default_graph()
checkpoint_path = './checkpoint/'
inference_path = './inference'
gan = GAN(get_hparas(), training_phase=True, ckpt_path=checkpoint_path, inference_path=inference_path,test_img = LRimgs)
gan.training()


build model started
[TL] InputLayer  VGG19/input: (?, 224, 224, 3)
[TL] Conv2d VGG19/conv1_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv1_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d VGG19/pool1: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d VGG19/conv2_1: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv2_2: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d VGG19/pool2: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d VGG19/conv3_1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv3_2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv3_3: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv3_4: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL

# Inference

In [13]:
checkpoint_path = './checkpoint/'
inference_path = './inference'
imgname = "000057_3"
tf.reset_default_graph()
gan = GAN(get_hparas(), training_phase=False, ckpt_path=checkpoint_path, inference_path=inference_path,test_img = LRimgs ,recover=60,savename = imgname)
img = gan.inference()

INFO:tensorflow:Restoring parameters from ./checkpoint/g_model_60.ckpt
-----success restored checkpoint--------
-----success save SRcrop--------
